In [87]:
import time
from tqdm import tqdm
import pandas as pd
from selenium import webdriver
import warnings

In [89]:
warnings.filterwarnings('ignore')

# 1. TRIPADVISOR 크롤링 

In [ ]:
base_url = "https://www.tripadvisor.co.kr/Restaurants-g983296-Jeju_Jeju_Island.html"
driver = webdriver.Chrome(executable_path='D:\Python_417\chromedriver')
driver.get(url=base_url)


def ta_first_step(raw_food, food_list, link_list):
    for food in raw_food:
        food_list.append(food.get_text())
        link_list.append(food['href'])
    return food_list, link_list

In [ ]:
driver.find_element_by_css_selector('a.nav.next.rndBtn.ui_button.primary.taLnk').click()
page_num = (2887 // 30) + 1;
page_num
food_list, link_list = [], []
for page in range(page_num):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    raw_food = soup.select('a.Lwqic.Cj.b')
    food_list, link_list = ta_first_step(driver, food_list, link_list)
    driver.find_element_by_css_selector('a.nav.next.rndBtn.ui_button.primary.taLnk').click()
    time.sleep(3)
final_df = pd.DataFrame()
final_df['name'] = food_list
final_df['link'] = link_list
final_df = final_df.drop_duplicates();
final_df

In [ ]:
def ta_second_step(soup, star_list, numreview_list, address_list, telephone_list, food_star_list, service_star_list,
                   price_star_list, price_list, menu_type_list):
    try:
        star = soup.select('span.ZDEqb')[0].get_text().strip()
    except:
        star = 'empty'
    try:
        numreview = soup.select('div.QEQvp > a.IcelI')[0].get_text()[:-5]
    except:
        numreview = 'empty'
    try:
        address = soup.select('span.DsyBj.cNFrA > span > a')[0].get_text()
    except:
        address = 'empty'
    try:
        telephone = soup.select('span.DsyBj.cNFrA > span > span.AYHFM > a')[0]['href']
    except:
        telephone = 'empty'
    # 각각의 평점
    try:
        food_star = soup.select('div.YDAvY.R2.F1.e.k > div > div > div.DzMcu > span.vzATR > span')[0]['class'][1][
                    7:]  # 음식
        service_star = soup.select('div.YDAvY.R2.F1.e.k > div > div > div.DzMcu > span.vzATR > span')[1]['class'][1][
                       7:]  # 서비스
        price_star = soup.select('div.YDAvY.R2.F1.e.k > div > div > div.DzMcu > span.vzATR > span')[2]['class'][1][
                     7:]  # 가격
    except:
        food_star, service_star, price_star = ['empty'] * 3
    try:
        if soup.select('div.tbUiL.b')[0].get_text() == '가격대':
            price = soup.select('div.SrqKb')[0].get_text()
            menu_type = soup.select('div.SrqKb')[1].get_text()
        elif soup.select('div.tbUiL.b')[0].get_text() == '요리':
            price = 'empty'
            menu_type = soup.select('div.SrqKb')[0].get_text()
        else:
            price, menu_type = ['empty'] * 2
    except:
        price, menu_type = ['empty'] * 2
    star_list.append(star)
    numreview_list.append(numreview)
    address_list.append(address)
    telephone_list.append(telephone)
    food_star_list.append(food_star)
    service_star_list.append(service_star)
    price_star_list.append(price_star)
    price_list.append(price)
    menu_type_list.append(menu_type)
    return star_list, numreview_list, address_list, telephone_list, food_star_list, service_star_list, price_star_list, price_list, menu_type_list

In [ ]:
star_list, numreview_list, address_list, telephone_list, food_star_list, service_star_list, price_star_list, price_list, menu_type_list = [], [], [], [], [], [], [], [], []
for link in final_df.link:
    driver.get(url=f'https://www.tripadvisor.co.kr{link}')
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    star_list, numreview_list, address_list, telephone_list, food_star_list, service_star_list, price_star_list, price_list, menu_type_list = ta_second_step(
        soup, star_list, numreview_list, address_list, telephone_list, food_star_list, service_star_list,
        price_star_list, price_list, menu_type_list)


In [ ]:
final_df['star'] = star_list
final_df['numreview'] = numreview_list
final_df['address'] = address_list
final_df['telephone'] = telephone_list
final_df['food_star'] = food_star_list
final_df['service_star'] = service_star_list
final_df['price_star'] = price_star_list
final_df['price'] = price_list
final_df['menu_type'] = menu_type_list
final_df.to_csv('TRIPADVISOR_식당크롤링.csv', encoding='utf-8-sig', index=False)

In [ ]:
def ta_hotel_step(raw_hotel, raw_price, hotel_list, link_list):
    for hotel, price in zip(raw_hotel, raw_price):
        hotel_list.append(hotel.select('a')[0].get_text().strip())
        link_list.append(hotel.select('a')[0]['href'])
        try:
            price_list.append(price.get_text())
        except:
            price_list.append('empty')
    return hotel_list, link_list, price_list

In [ ]:
page_num = 2048 // 30 + 1
hotel_list, link_list, price_list = [], [], []
for page in tqdm(range(page_num)):
    driver.implicitly_wait(30)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    raw_hotel = soup.select('div.listing-title')
    raw_price = soup.select('div.price-wrap > div.price.__resizeWatch')
    hotel_list, link_list, price_list = ta_hotel_step(raw_hotel, raw_price, hotel_list, link_list)
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollheight);')
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    driver.find_element_by_css_selector('a.nav.next.ui_button.primary').click()
    time.sleep(5)


In [ ]:
final_df = pd.DataFrame()
final_df['name'] = hotel_list
final_df['link'] = link_list
final_df['price'] = price_list
final_df = final_df.drop_duplicates().reset_index(drop=True)
final_df.to_csv('TRIPADVISOR_숙소크롤링.csv', encoding='utf-8-sig', index=False)

# 2. MYREALTRIP 크롤링

In [ ]:
base_url = "https://www.myrealtrip.com/accommodations/stays/?place[place_type]=GEO&place[place_name]=%EC%A0%9C%EC%A3%BC%EB%8F%84&checkin=2022-10-04&checkout=2022-10-07&adult_count=3&order_by=WEIGHT_DESC&page[page]=1&page[size]=30&filter[min_price]=0&filter[max_price]=1000000&filter[star]=0&filter[only_dgm_promo]=false&filter[curation_type]=ALL"
driver = webdriver.Chrome(executable_path='D:\Python_417\chromedriver')
driver.get(url=base_url)


def mt_crawling(driver):
    for link in driver.find_elements_by_css_selector('#Mrt3Stay > main > div > section > div > a'):
        link_list.append(link.get_attribute('href'))

    for name in driver.find_elements_by_css_selector(
            '#Mrt3Stay > main > div > section > div > a > div.css-4tl1r5--ProductCard--Content.e9e97mf3 > h2'):
        name_list.append(name.text)

    for address in driver.find_elements_by_css_selector(
            '#Mrt3Stay > main > div > section > div > a > div.css-4tl1r5--ProductCard--Content.e9e97mf3 > div.css-525l10--ProductCard--Infos.e9e97mf1 > p'):
        address_list.append(address.text.split('・')[-1].strip())

    for price in driver.find_elements_by_css_selector(
            '#Mrt3Stay > main > div > section > div > a > div.css-4tl1r5--ProductCard--Content.e9e97mf3 > div.css-ck4a4o--ProductPrice--Container.e8oi5wt5 > div.stay-product-price-saleprice.css-bdcqse--ProductPrice--Wrapper.e8oi5wt4 > p.css-s4tmwy--ProductPrice--AveragePrice.e8oi5wt0'):
        price_list.append(price.text.split('1박')[-1])

    for star in driver.find_elements_by_css_selector(
            '#Mrt3Stay > main > div > section > div> a > div.css-4tl1r5--ProductCard--Content.e9e97mf3 > div.css-525l10--ProductCard--Infos.e9e97mf1 > div'):
        tmp = star.text.split('\n')[0]
        if len(tmp) == 1:
            star_list.append('empty')
            star_ppl_list.append('empty')
        else:
            star_list.append(tmp[0])
            star_ppl_list.append(tmp[1])

In [ ]:
num_page = (1025 // 30) + 1
link_list, name_list, address_list, price_list, star_list, star_ppl_list = [], [], [], [], [], []
for page in tqdm(range(num_page)):
    mt_crawling(driver)
    driver.find_element_by_css_selector(
        '#Mrt3Stay > main > div > section > div.e9wuk890.css-f35ifk--ProductList--Pagination > div > li:nth-child(7) > button').click()
    driver.implicitly_wait(10)
    time.sleep(5)

In [ ]:
myrealtrip = pd.DataFrame([link_list, name_list, address_list, price_list, star_list, star_ppl_list]).T
myrealtrip.columns = ['link', 'name', 'address', 'price', 'star', 'star_ppl']
myrealtrip['price'] = myrealtrip['price'].apply(lambda x: int(x.replace(',', '').replace("원", "").strip()))
myrealtrip['price'] = myrealtrip['price'].apply(lambda x: x * 3)
myrealtrip.to_csv('MYREALTRIP_숙소크롤링.csv', encoding='utf-8-sig', index=False)